# MakiFlow usage examples

This notebook contains basic guidelines of using MakiFlow. MakiFlow is not a serious framework like PyTorch or TensorFlow, this is an instrument I built in order to improve quality and speed of my researches in the field of Neural Networks, especially CNNs. If you found this library useful and have some thoughs how to improve it or what to fix I'd like to listen to you. My email: igor.kilbas@mail.ru.

## Importing MakiFlow

As far as I don't know how to create high quality libraries, you're gonna have some pain. Here is the way I import MakiFlow on my computer:

In [2]:
import sys
sys.path.append('/home/student401/MakiFlow')

# Creating CNN based on MakiFlow

Now you have an access to MakiFlow and can import stuff related to it.
In order to create CNN you need to defined layers order first. Here is what I mean by that:

In [3]:
from makiflow.layers import ConvLayer, DenseLayer, MaxPoolLayer, FlattenLayer

layers = [
        ConvLayer(kw=3, kh=3, in_f=3, out_f=64, name='input_layer'),
        ConvLayer(kw=3, kh=3, in_f=64, out_f=64, name=2),
        MaxPoolLayer(),
    
        ConvLayer(kw=3, kh=3, in_f=64, out_f=128, name=4),
        ConvLayer(kw=3, kh=3, in_f=128, out_f=128, name=5),
        MaxPoolLayer(),
    
        ConvLayer(kw=3, kh=3, in_f=128, out_f=256, name=6),
        ConvLayer(kw=3, kh=3, in_f=256, out_f=256, name=7),
        MaxPoolLayer(),
    
        ConvLayer(kw=3, kh=3, in_f=256, out_f=512, name=8),
        ConvLayer(kw=3, kh=3, in_f=512, out_f=512, name=9),
        ConvLayer(kw=3, kh=3, in_f=512, out_f=512, name=10),
        MaxPoolLayer(),
    
        FlattenLayer(),
        DenseLayer(input_shape=2048, output_shape=1024, name=12),
        DenseLayer(input_shape=1024, output_shape=1024, name=13),
        DenseLayer(input_shape=1024, output_shape=10, activation=None, name='out_put_layer')
    # The last layer always is gonna have no activation function! Just always pass None into 'activation' argument!
    ]

We have built basic VGG model. As you might have noticed, you have to directly pass the number of input feature maps and output feature maps, so be careful with that. Tip: number of the input feature maps in the next layer equals number of the feature maps in the previous layer. Anyway, TensorFlow will get you know if something is wrong.

To create CNN, we need ConvModel class:

In [5]:
from makiflow import ConvModel

model = ConvModel(layers=layers, input_shape=[64, 32, 32, 3], output_shape=[64, 10], name='My_MakiFlow_little_VGG')

That's it! You've just created your first CNN based on MakiFlow! But we still have some stuff to do.

First, ConvModel requires TensorFlow Session in order to work. You can treat ConvModel as a car and Session as an engine. Let's add an engine to our car:

In [7]:
import tensorflow as tf

session = tf.Session()
model.set_session(session)

# Training CNN

Now we can use the model. Let's train it. We'll use cifar10 dataset for this purpose.

In [9]:
import numpy as np
import keras
from keras.datasets import cifar10

(Xtrain, Ytrain), (Xtest, Ytest) = cifar10.load_data()
    
Xtrain = Xtrain.astype(np.float32)
Xtest = Xtest.astype(np.float32)

Xtrain /= 255
Xtest /= 255

# Convert class vectors to binary class matrices.
Ytrain = keras.utils.to_categorical(Ytrain, 10)
Ytest = keras.utils.to_categorical(Ytest, 10)

Second step is to define our training parameters:
learning_rate;
optimizer - ConvModel uses TensorFlow optimizers in order to train the model;
epochs;

We will use RMSProp for training the model.

In [11]:
epochs = 2
lr = 1e-5*128
epsilon = 1e-6
optimizer = tf.train.RMSPropOptimizer(learning_rate=lr, epsilon=epsilon)

In order to train the model we can call two methods:
verbose_fit - tests the model on train data and test data;
pure_fit - tests the model only on test data, it works much faster than verbose_fit.

We will use pure fit.

In [12]:
info = model.pure_fit(Xtrain, Ytrain, Xtest, Ytest, optimizer=optimizer, epochs=epochs)

  0%|          | 0/781 [00:00<?, ?it/s]

Epoch: 0 Train accuracy: 0.2597880006723787 Train cost: 1.9747080735386964 Test accuracy 0.22870000000000001 Test cost 2.4128412099984975


100%|██████████| 781/781 [00:12<00:00, 61.63it/s]


Epoch: 1 Train accuracy: 0.4819966682592617 Train cost: 1.418893301204675 Test accuracy 0.5457000000000001 Test cost 1.2490314581455328


pure_fit method returns dictionary with tests' info on each epoch.

In [13]:
info

{'train costs': [1.9747080735386964, 1.418893301204675],
 'train errors': [0.7402119993276213, 0.5180033317407383],
 'test costs': [2.4128412099984975, 1.2490314581455328],
 'test errors': [0.7713, 0.4543]}

# Making predictions using MakiFlow model

In order to make predictions, we use predict method. 
This method requires tensor of shape (number_of_samples, dim, dim, channels).
It always returns numpy array with probabilities of belonging this particular sample(or samples) to each class.

In [15]:
model.predict(Xtest[0:1])

array([[0.01108474, 0.00861162, 0.12099435, 0.36020404, 0.06199497,
        0.27742323, 0.10490333, 0.03577648, 0.01214113, 0.00686607]],
      dtype=float32)